References
- https://www.kaggle.com/code/huikang/arc-agi-2-code-approach
- https://www.kaggle.com/code/huikang/r1-distill-qwen-tir

In [1]:
import subprocess

subprocess.run(
    ["pip", "uninstall", "--yes", "tensorflow", "matplotlib", "keras", "scikit-learn"]
)

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: matplotlib 3.7.2
Uninstalling matplotlib-3.7.2:
  Successfully uninstalled matplotlib-3.7.2
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2


CompletedProcess(args=['pip', 'uninstall', '--yes', 'tensorflow', 'matplotlib', 'keras', 'scikit-learn'], returncode=0)

In [2]:
import os
import time
import torch
import numpy as np


def is_on_kaggle_commit() -> bool:
    return os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Batch" and not bool(
        os.getenv("KAGGLE_IS_COMPETITION_RERUN")
    )


def is_on_kaggle_interactive() -> bool:
    return os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Interactive" and not bool(
        os.getenv("KAGGLE_IS_COMPETITION_RERUN")
    )


start_time = time.time()
final_cutoff_time = start_time + (4 * 60 + 45) * 60  # 4.75 hours from start time
cutoff_times = [
    int(x) for x in np.linspace(final_cutoff_time, start_time + 5 * 60, 50 + 1)
]  # 5 minutes loading time at the start
cutoff_times.pop()

os.makedirs("solutions", exist_ok=True)

assert torch.cuda.is_available()

# Serve vLLM

In [3]:
subprocess.run(["ls", "/kaggle/usr/lib/pip_install_aimo3/tiktoken_encodings"])

cl100k_base.tiktoken
o200k_base.tiktoken


CompletedProcess(args=['ls', '/kaggle/usr/lib/pip_install_aimo3/tiktoken_encodings'], returncode=0)

In [4]:
import subprocess


def start_vllm_server() -> subprocess.Popen[bytes]:
    """Start vLLM server in the background"""
    os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    # https://docs.vllm.ai/projects/recipes/en/latest/OpenAI/GPT-OSS.html#troubleshooting
    os.environ["TIKTOKEN_ENCODINGS_BASE"] = (
        "/kaggle/usr/lib/pip_install_aimo3/tiktoken_encodings"
    )

    sequence_length = 65_536  # 32 * 2048

    command: list[str] = [
        "python",
        "-m",
        "vllm.entrypoints.openai.api_server",
        "--model",
        "/kaggle/input/gpt-oss-20b/transformers/default/1",
        "--served-model-name",
        "qwen3",
        "--tensor-parallel-size",
        "1",
        "--host",
        "0.0.0.0",
        "--port",
        "8000",
        "--dtype",
        "auto",
        "--max-model-len",
        f"{sequence_length}",
        "--max-seq-len-to-capture",
        f"{sequence_length}",
    ]

    # Start the process in the background
    with open("/kaggle/working/vllm.log", "w") as logfile:
        process: subprocess.Popen[bytes] = subprocess.Popen(
            command, stdout=logfile, stderr=subprocess.STDOUT, start_new_session=True
        )

    print("Logs: /kaggle/working/vllm.log")
    return process


# Start the server
vllm_process: subprocess.Popen[bytes] = start_vllm_server()

Logs: /kaggle/working/vllm.log


In [5]:
from openai import OpenAI, Stream
from openai.types.chat import ChatCompletion, ChatCompletionMessageParam
from openai.types.chat.chat_completion_chunk import ChatCompletionChunk

# Point the client to your local vLLM server
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8000/v1"
os.environ["OPENAI_API_KEY"] = "sk-local"  # any non-empty string

client: OpenAI = OpenAI(
    base_url=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"],
)

stop_token_ids = [
    token_id
    for token_id in range(200_000, 201_088)
    if token_id not in [200005, 200006, 200007, 200008]
]

In [6]:
import time

for _ in range(15 * 60):
    time.sleep(1)
    try:
        print(client.models.list())
    except Exception:
        continue
    break
else:
    raise ValueError

SyncPage[Model](data=[Model(id='qwen3', created=1763884103, object='model', owned_by='vllm', root='/kaggle/input/gpt-oss-20b/transformers/default/1', parent=None, max_model_len=65536, permission=[{'id': 'modelperm-2b2094fe6ce14ac2a80ab58ef11e800a', 'object': 'model_permission', 'created': 1763884103, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')


In [7]:
from cachetools import cached, TTLCache
from typing import Generator
import time

import os
import re


def reversed_lines(path: str, block_size: int = 4096) -> Generator[str, None, None]:
    """
    Iterate over the lines of a file in reverse order (last line first),
    without loading the entire file into memory.

    Yields lines as strings (including the trailing newline if present).
    """
    with open(path, "rb") as f:
        f.seek(0, os.SEEK_END)
        file_end = f.tell()

        buffer = b""
        pos = file_end

        while pos > 0:
            # Read a block from the end going backwards
            read_size = min(block_size, pos)
            pos -= read_size
            f.seek(pos, os.SEEK_SET)
            data = f.read(read_size)

            buffer = data + buffer
            # Split into lines
            lines = buffer.split(b"\n")
            # Keep the first (possibly incomplete) part in buffer
            buffer = lines[0]
            # The rest (from the end backwards) are full lines
            for line in reversed(lines[1:]):
                yield line.decode("utf-8", errors="replace") + "\n"

        # Finally, yield the very first line (if any)
        if buffer:
            yield buffer.decode("utf-8", errors="replace") + "\n"


@cached(cache=TTLCache(maxsize=2, ttl=10))
def get_gpu_kv_cache_usage() -> float:
    for line in reversed_lines("vllm.log"):
        pattern = r"GPU KV cache usage: ([\d.]+)%"
        match = re.search(pattern, line)
        if match:
            gpu_cache_usage = float(match.group(1))
            return gpu_cache_usage
    return 0

In [8]:
resp: ChatCompletion = client.chat.completions.create(
    model="qwen3",  # use your served name; if not set, the model path/name vLLM shows in logs
    messages=[
        {"role": "system", "content": "Reply your answer in \\boxed{}"},
        {"role": "user", "content": "How many r are there in strawberry?"},
    ],
    max_tokens=1024,
    temperature=1.0,
    extra_body=dict(min_p=0.02, stop_token_ids=stop_token_ids),
)

In [9]:
print(resp.choices[0].message.reasoning_content)

User asks: "How many r are there in strawberry?" Likely they mean count the letter 'r' in the word "strawberry". The word is s t r a w b e r r y. The 'r's: positions 3, 8, 9? Let's list: s(1), t(2), r(3), a(4), w(5), b(6), e(7), r(8), r(9), y(10). So there are 3 r's. But might consider "strawberries" plural? But question says strawberry. So answer: 3. And need to reply in \boxed{3}.


In [10]:
print(resp.choices[0].message.content)

\boxed{3}


# Prediction

In [11]:
def extract_boxed_text(text: str) -> str:
    """Extract text inside \\boxed{} from LaTeX-formatted text"""
    import re

    pattern: str = r"oxed{(.*?)}"
    matches: list[str] = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""


def is_valid_answer_string(text: str) -> bool:
    try:
        if int(text) == float(text):
            if 0 <= int(text) <= 99_999:
                # now AIMO answers no longer need modulo
                return True
    except Exception:
        pass
    return False

In [12]:
from collections import Counter

completed_question_ids: set[str] = set()
question_id_to_counter: dict[str, Counter] = {"": Counter()}


import math
from collections import Counter


def vote_answer(question_id: str, force_answer: bool = False) -> int | None:
    # reads counter from global
    counter = question_id_to_counter[question_id]
    if force_answer and not counter:
        completed_question_ids.add(question_id)
        return 12453

    # voting mechanism
    modified_counter = Counter()
    for value, count in counter.items():
        # re-weighted because smaller answers seems to be wrong
        # "1.25 +" because log(1) = 0
        modified_counter[value] += math.log(1.25 + abs(value)) * count

    total_score = sum(modified_counter.values())
    score_list = sorted(
        (score, counter[value], value) for value, score in modified_counter.items()
    )
    if force_answer:
        print(f"score_list | {total_score:8.1f} over {sum(counter.values())} attempts")
        print(f"Current GPU usage {get_gpu_kv_cache_usage()}")
        for score, count, value in score_list[::-1]:
            print(f"{value:10}   {score:8.1f} {count:8d}")
        return score_list[-1][-1]
    if score_list[-1][0] > max(50, total_score / (2 + math.log(1 + total_score))):
        if len(score_list) == 1:
            completed_question_ids.add(question_id)
        else:
            if score_list[-1][0] - score_list[-2][0] > 50:
                # win by a certain number of points at least
                completed_question_ids.add(question_id)
    return None

In [13]:
import time


def generate_solution(
    question_text: str, question_id: str = "", solution_index: int = 0
) -> str:
    if question_id in completed_question_ids:
        return ""
    if time.time() >= cutoff_times[-1]:
        return ""
    if get_gpu_kv_cache_usage() > 65:
        time.sleep(1)  # otherwise we will flush the queue, which is unintended
        return ""

    messages: list[ChatCompletionMessageParam] = [
        {
            "role": "system",
            "content": "You will solve the problem and return the final answer in \\boxed{}. The answer is expected to be an integer between 0 and 99999 inclusive. Do not guess the answer.",
        },
        {"role": "user", "content": question_text},
    ]

    text_response_to_save = ""
    for iteration in range(3):
        text_response = ""
        breaking = False

        stream: Stream[ChatCompletionChunk] = client.chat.completions.create(
            model="qwen3",  # use your served name; if not set, the model path/name vLLM shows in logs
            messages=messages,
            temperature=1.0,
            stream=True,
            extra_body=dict(min_p=0.02, stop_token_ids=stop_token_ids),
        )

        for chunk in stream:
            chunk_text = (
                chunk.choices[0].delta.reasoning_content
                if hasattr(chunk.choices[0].delta, "reasoning_content")
                else chunk.choices[0].delta.content
            )
            if chunk_text:
                text_response += chunk_text
            # avoid stalling the GPU
            if get_gpu_kv_cache_usage() > 85 and solution_index % 10 == 1:
                print(
                    f"generation stopped to avoid excessive GPU usage, currently {get_gpu_kv_cache_usage()}"
                )
                breaking = True
            if get_gpu_kv_cache_usage() > 95:
                print(
                    f"generation due to excessive GPU usage, currently {get_gpu_kv_cache_usage()}"
                )
                breaking = True
            if question_id in completed_question_ids:
                # stop generating if we have finalized on an answer
                breaking = True
            if time.time() >= cutoff_times[-1]:
                breaking = True
            if breaking:
                break

        messages.append({"role": "assistant", "content": text_response})
        text_response_to_save += text_response
        stream.close()

        if breaking:
            break

        boxed_text = extract_boxed_text(text_response)
        if not is_valid_answer_string(boxed_text):
            user_follow_up = "The answer is expected to be an integer between 0 and 99999 inclusive. Place your final answer in \\boxed{}. Do not guess the answer."
            messages.append({"role": "user", "content": user_follow_up})
            text_response_to_save += "\n===\n" + user_follow_up + "\n===\n"
        elif int(boxed_text) <= 10:
            user_follow_up = "Are you sure that is the answer? Do not guess the answer."
            messages.append({"role": "user", "content": user_follow_up})
            text_response_to_save += "\n===\n" + user_follow_up + "\n===\n"
        elif iteration == 0 and get_gpu_kv_cache_usage() < 20:
            user_follow_up = "Have you verified your answer?"
            messages.append({"role": "user", "content": user_follow_up})
            text_response_to_save += "\n===\n" + user_follow_up + "\n===\n"
        else:
            # answer found, no issues detected, continuing
            break

    boxed_text = extract_boxed_text(
        text_response_to_save
    )  # expected to use the full conversation

    if question_id and text_response_to_save:
        with open(f"solutions/{question_id}/{solution_index:04d}.txt", "w") as f:
            f.write(text_response_to_save)

    if is_valid_answer_string(boxed_text):
        question_id_to_counter[question_id][int(boxed_text)] += 1
        vote_answer(question_id)

    return boxed_text

In [14]:
generate_solution("What is 1+1?")

'2'

In [15]:
import concurrent.futures
from collections import Counter


def solve(question_text: str, question_id: str = "") -> int:
    os.makedirs(f"solutions/{question_id}", exist_ok=True)
    question_id_to_counter[question_id] = Counter()
    completed_question_ids.discard(question_id)

    if question_id and time.time() > cutoff_times[-1]:
        print("timeout did not solve")
        return 12314

    num_generations = 1024
    get_gpu_kv_cache_usage()  # run once to prevent running in the first batch of execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=64) as executor:
        # run in parallel
        results = executor.map(
            generate_solution,
            [question_text] * num_generations,
            [question_id] * num_generations,
            list(range(num_generations)),
        )
        list(results)

    final_answer = vote_answer(question_id, force_answer=True)
    return final_answer

In [16]:
solve("What is 1+1?")

score_list |    115.5 over 98 attempts
Current GPU usage 0
         2      115.5       98


2

# Submission

In [17]:
import os

import kaggle_evaluation.aimo_3_inference_server
import pandas as pd
import polars as pl

# 本地 / notebook 调试用的测试集路径
TEST_CSV_PATH = "/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv"

# Replace this function with your inference code.
# The function should return a single integer between 0 and 99999, inclusive.
def predict(id_: pl.Series, problem: pl.Series) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # Unpack values（Polars 输入是一列，这里取单个样本）
    question_id = id_.item(0)
    problem_text: str = problem.item(0)

    # 调用你前面定义好的 solve()，里面已经封装了多次采样、投票等逻辑
    prediction = solve(problem_text, question_id=str(question_id))

    # 和你原来的代码保持一致的全局状态更新
    completed_question_ids.add(str(question_id))
    cutoff_times.pop()

    # AIMO3 要求返回一个 DataFrame，列名必须是 id / answer
    return pl.DataFrame({"id": question_id, "answer": prediction})


# 和官方 demo 一样的推理服务包装
inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(
    predict
)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    # 评测环境走这里：启动真正的 gRPC 服务，让官方评测脚本来喂题目
    # 注意：你必须在脚本启动 15 分钟内调用 serve()，否则会被判失败
    inference_server.serve()
else:
    # 本地 / Notebook 调试：直接用 test.csv 跑一遍，模拟评测
    inference_server.run_local_gateway((TEST_CSV_PATH,))


score_list |     61.1 over 274 attempts
Current GPU usage 0.7
         0       61.1      274
score_list |     61.4 over 275 attempts
Current GPU usage 0.3
         0       61.4      275
score_list |     61.4 over 275 attempts
Current GPU usage 0.8
         0       61.4      275
